# VacationPy
Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sitka,57.0531,-135.3300,280.53,70,100,1.01,US,1698613202
1,1,tiksi,71.6872,128.8694,257.66,99,92,2.88,RU,1698613206
2,2,blackmans bay,-43.0167,147.3167,287.22,60,93,1.34,AU,1698613211
3,3,morondava,-20.2833,44.2833,298.15,74,13,4.71,MG,1698613359
4,4,korsakov,46.6328,142.7813,269.06,71,14,1.23,RU,1698613215


Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600    
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_cities = (city_data_df['Cloudiness'] >= 80) & (city_data_df['Lat'] >= 0)

# Filter DataFrame by criteria
city_data_df_filtered = city_data_df[narrowed_cities]

# Drop any rows with null values
city_data_df_filtered = city_data_df_filtered.dropna()

# Display sample data
city_data_df_filtered.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sitka,57.0531,-135.3300,280.53,70,100,1.01,US,1698613202
1,1,tiksi,71.6872,128.8694,257.66,99,92,2.88,RU,1698613206
6,6,whitehorse,60.7161,-135.0538,270.94,93,100,6.17,CA,1698613219
15,15,albany,42.6001,-73.9662,280.11,92,100,1.61,US,1698613095
18,18,mana,5.6592,-53.7767,300.90,81,99,6.56,GF,1698613437


Step 3: Create a new DataFrame called hotel_df

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_filtered.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sitka,US,57.0531,-135.3300,70,
1,tiksi,RU,71.6872,128.8694,99,
6,whitehorse,CA,60.7161,-135.0538,93,
15,albany,US,42.6001,-73.9662,92,
18,mana,GF,5.6592,-53.7767,81,
...,...,...,...,...,...,...
521,ribeira brava,PT,32.6500,-17.0667,81,
533,pommelsbrunn,DE,49.5044,11.5110,89,
535,oak hill,US,37.9723,-81.1487,72,
538,mizan teferi,ET,6.9833,35.5833,98,


Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000

params = {
    "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
   
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"
   

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sitka - nearest hotel: Aspen Suites Hotel
tiksi - nearest hotel: Арктика
whitehorse - nearest hotel: Town & Mountain Hotel
albany - nearest hotel: No hotel found
mana - nearest hotel: Le Samana
weno - nearest hotel: High Tide Hotel
nar'yan-mar - nearest hotel: Гостиница "Агат"
chiang khong - nearest hotel: Chiang Khong Green Inn
iskateley - nearest hotel: Авантаж
mandera - nearest hotel: Kornesh Hotel and Lodging
port-cartier - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
sinabang - nearest hotel: No hotel found
inirida - nearest hotel: Residencias El Turista del Guainia
mitu - nearest hotel: Hotel Brasilia Vaupés
ceeldheer - nearest hotel: No hotel found
twon-brass - nearest hotel: KC Hotel
yesilyurt - nearest hotel: Ramda Plaza Altınkayısı Otel
dakhla - nearest hotel: Hotel El Massira
khandyga - nearest hotel: Апельсин
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
superior - nearest hotel: Androy Hotel
tralee - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sitka,US,57.0531,-135.3300,70,Aspen Suites Hotel
1,tiksi,RU,71.6872,128.8694,99,Арктика
6,whitehorse,CA,60.7161,-135.0538,93,Town & Mountain Hotel
15,albany,US,42.6001,-73.9662,92,No hotel found
18,mana,GF,5.6592,-53.7767,81,Le Samana
...,...,...,...,...,...,...
521,ribeira brava,PT,32.6500,-17.0667,81,Cheerfulway Bravamar Hotel
533,pommelsbrunn,DE,49.5044,11.5110,89,Lindenhof
535,oak hill,US,37.9723,-81.1487,72,Holiday Lodge
538,mizan teferi,ET,6.9833,35.5833,98,ሆቴል ሳላይሽ / Salayesh Hotel


Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    hover_cols= ["Hotel Name", "Country"],
    frame_width = 800,
    frame_height = 600    
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)